### Content-Based Recommendation System

<p>Sistema simples de recomendação de filmes. Usando o tipo filtragem baseada em conteúdo.</p> 

#### Limpeza e exploração dos dados

In [2]:
# Importação das Bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("datasets/movies.csv")

df.info()

In [ ]:
df = df[['original_title', 'production_companies', 'genres', 'overview']]

df[['production_companies', 'genres']] 

<p>No dataset importado tem colunas que contém dicionários dentro de lista, então precisa ser tratados</p>

In [6]:
def extraction(dataframe, column: str, new_column_name: str):
    # Extraí dados de dicionários
    data_for_extraction = dataframe[column]
    genre_lists = [list(set([genre['name'] for genre in eval(item)])) for item in data_for_extraction]
    dataframe[new_column_name] = genre_lists

In [7]:
extraction(df, 'genres', 'new_genres')
extraction(df, 'production_companies', 'new_companies')

In [8]:
df = df.drop(labels=['production_companies', 'genres'], axis=1)

In [9]:
df['info'] = df['new_genres'].astype(str) + df['new_companies'].astype(str) + df['overview'].astype(str)
df['info'] = df['info'].str.replace(r"\[", "", regex=True).str.replace(r"\]", "", regex=True).str.replace("'", "", regex=True).str.replace(" ", "", regex=True)

In [ ]:
df.head()

#### Pré-processamento de dados

In [11]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['info'].apply(lambda x: np.str_(x)))

In [ ]:
print(f'Matriz TF-IDF: {tfidf_matrix.toarray()}')

In [13]:
similarity = cosine_similarity(tfidf_matrix)

In [ ]:
similarity_matrix = pd.DataFrame(similarity, columns=df['original_title'], index=df['original_title'])

similarity_matrix

In [ ]:
df['original_title'].sample(10) # Encontra titúlos

In [47]:
movie_recommendation = pd.DataFrame(similarity_matrix["X-Men"].sort_values(ascending=False))
movie_recommendation.columns = ['Top 10 Recomendações']

In [ ]:
movie_recommendation.head(10)